In [0]:
import pandas as pd
import random
import io
import ast
import numpy as np
import os
import re
import string
import pickle
import copy
import seaborn as sns
import torch.nn.functional as functional
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from bs4 import BeautifulSoup
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import nltk
from nltk.stem import WordNetLemmatizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import pairwise_distances

from time import time
from tqdm import tnrange

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Loading texts, split on train, val and test


In [0]:
!wget https://www.dropbox.com/s/n8nfxxh5azocrrm/train_LASER.csv?dl=0
!wget https://www.dropbox.com/s/je9am5c77ytfcaf/test_LASER.csv?dl=0 

In [0]:
#открываем файл с train dataset

df_train = pd.read_csv("train_LASER.csv?dl=0").dropna()

train_texts = list(df_train['abstracts'])
train_labels = list(df_train['labels'])
len(train_labels)


80948

In [0]:
#открываем файл с test_dataset

df_test = pd.read_csv("test_LASER.csv?dl=0").dropna()

test_texts = list(df_test['lemm_abstracts'])
test_labels = list(df_test['labels'])
len(test_labels)

10150

In [0]:
#разбиваем на train и val английские тексты
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, \
                                                          test_size=0.15, shuffle = True, random_state=42, stratify=train_labels)

In [0]:
print(len(train_labels))
print(len(val_labels))

68805
12143


In [0]:
test_texts[1000]

'использование биотехнология, медицинская микробиология и может быть использовано в производстве эубиотических препаратов и бифидосодержащих продуктов питания. сущность изобретения питательная среда в качестве основы содержит молочную сыворотку, осветленную последовательной обработкой обезжиренного молока кислотой и щелочью, и дрожжевой автолизат сухой или жидкий и дополнительно натрий фосфорнокислый однозамещенный двуводный, аммоний хлористый и магний сернокислый семиводный, при следующем соотношении компонентов мас. молочная сыворотка осветленная , , , лактоза , , , дрожжевой автолизат сухой или жидкий , , и , , , d,l цистеин , , , nah po • h o , , , nh cl , , , mgso • h o , , , агар агар , , , вода остальное, ph , , .'

In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
#add_labels = label_encoder.fit_transform(df_add['us_code'])
y_train[:10]

array([ 58, 199, 218,   7, 196, 115,  69,   1,  13,  23])

In [0]:
y_val = label_encoder.transform(val_labels)
y_val[:10]

array([ 23,  82,  57, 164,  99, 146,  24, 161, 194,  16])

In [0]:
y_test = label_encoder.transform(test_labels)

In [0]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(f'Using device: {DEVICE}')

Using device: cuda


### Cleaning

In [0]:
def cleaning(text):
    clean_txt = []
    for sentence in text.split('.'): 
        sentence = re.sub(r'[•⋅−,]', ' ', sentence)
        sent = re.sub(r'( )+', ' ', sentence)
     #   if len(sent.split()) > 2 and sent != 'ф лы, ил':
        if len(sent.split()) > 2 and 'ф лы' not in sent:
             clean_txt.append(sent.strip())
    return ' '.join(clean_txt) 

In [0]:
test_texts = [cleaning(text) for text in tqdm(test_texts)]
val_texts = [cleaning(text) for text in tqdm(val_texts)]
train_texts = [cleaning(text) for text in tqdm(train_texts)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
test_texts[1000]

'использование биотехнология медицинская микробиология и может быть использовано в производстве эубиотических препаратов и бифидосодержащих продуктов питания сущность изобретения питательная среда в качестве основы содержит молочную сыворотку осветленную последовательной обработкой обезжиренного молока кислотой и щелочью и дрожжевой автолизат сухой или жидкий и дополнительно натрий фосфорнокислый однозамещенный двуводный аммоний хлористый и магний сернокислый семиводный при следующем соотношении компонентов мас молочная сыворотка осветленная лактоза дрожжевой автолизат сухой или жидкий и d l цистеин nah po h o nh cl mgso h o агар агар вода остальное ph'

In [0]:
def compute_lens(texts):
    lens = [len(text.split()) for text in texts]
    return lens

train_lens = compute_lens(train_texts)
val_lens = compute_lens(val_texts)
test_lens = compute_lens(test_texts)
train_lens[:10]

[142, 194, 152, 77, 98, 100, 149, 206, 96, 158]

### Lemmatization

In [0]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-04-28 11:35:38--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.245, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm910.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-04-28 11:35:38--  http://cache-mskm910.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm910.cdn.yandex.net (cache-mskm910.cdn.yandex.net)... 5.45.220.20, 2a02:6b8:0:2002::1b
Connecting to cache-mskm910.cdn.yandex.net (cache-mskm910.cdn.yandex.net)|5.45.220.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [0]:
from pymystem3 import Mystem
m = Mystem()

In [0]:
test_lemm = [''.join(m.lemmatize(text)).strip() for text in tqdm(test_texts)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [0]:
import spacy
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en', disable=['parser', 'ner'])

train_lemm = []
for text in tqdm(train_texts):
    train_lemm.append(" ".join([w.lemma_ if w.lemma_ !='-PRON-' else w.text for w in nlp(text)]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [0]:
val_lemm = []
for text in tqdm(val_texts):
    val_lemm.append(" ".join([w.lemma_ if w.lemma_ !='-PRON-' else w.text for w in nlp(text)]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


### Loading MUSE embeddings

In [0]:
def load_vec(emb_path, nmax=200000):   #200000 - это максимальное число слов, которое есть в MUSE embeddings
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [0]:
#download MUSE russian embeddings
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.ru.vec

#download MUSE english embeddings
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec

--2020-04-28 11:45:31--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.ru.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 630591038 (601M) [text/plain]
Saving to: ‘wiki.multi.ru.vec’

wiki.multi.ru.vec   100%[===================>] 601.38M  11.6MB/s    in 53s     

2020-04-28 11:46:25 (11.3 MB/s) - ‘wiki.multi.ru.vec’ saved [630591038/630591038]

--2020-04-28 11:46:26--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628614720 (599M) [text/plain]
Saving to: ‘wiki.multi.e

In [0]:
src_embeddings, src_id2word, src_word2id = load_vec('wiki.multi.en.vec')
tgt_embeddings, tgt_id2word, tgt_word2id = load_vec('wiki.multi.ru.vec')

### Classification by keywords

In [0]:
def matrix_for_features(texts, labels):
    '''
    функция для поиска ключевых слов для каждого класса
    подаются тексты и метки
    возвращается матрица tfidf и фичи
    '''
    if len(labels) > 0:
        text_classes = ['' for i in range(len(set(labels)))]
        for i, text in enumerate(texts):
        #  text_classes[labels[i]] += (' '.join(list(set(text.split()))) + ' ')
            text_classes[labels[i]] += (text + ' ')
        vectorizer = TfidfVectorizer(min_df=2, max_df=0.8)
        X = vectorizer.fit_transform(text_classes)
    else:
        vectorizer = TfidfVectorizer(min_df=2, max_df=0.8)
        X = vectorizer.fit_transform(texts)
    feature_names = np.array(vectorizer.get_feature_names())
    return X, feature_names

In [0]:
src_emb_norm = src_embeddings / np.linalg.norm(src_embeddings, 2, 1)[:, None]
def get_similar(words, src_emb_norm, tgt_emb, tgt_id2word, K=1):
    '''
    подаётся слово на русском
    выдаются индексы K ближайших векторов на английском
    '''
    words_emb = np.array([tgt_emb[tgt_word2id[word]] for word in words])
    words_emb_norm = words_emb/np.linalg.norm(words_emb, 2, 1)[:, None]
 #   scores = (src_emb / np.linalg.norm(src_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    scores = pairwise_distances(words_emb_norm, src_emb_norm, metric='cosine')
 #   print(scores.argsort().shape)
    K_best = scores.argsort()[:,:K]
  #  for i, idx in enumerate(one_best):
    return K_best

In [0]:
idx = get_similar(['электричество', 'университет'], src_emb_norm, tgt_embeddings, tgt_id2word, K=1)
idx

array([4424,   98])

In [0]:
np.mean(src_embeddings[idx], axis = 1).shape

(2, 300)

In [0]:
X_russian, features_russian = matrix_for_features(test_lemm, labels=y_test)
X_english, features_english = matrix_for_features(train_lemm, labels=y_train)

In [0]:
def top_keywords_classes(tfidf_matrix, features, K=15):
  for i in range(225):
    tfidf = tfidf_matrix.toarray()[0]
    indices = np.argsort(tfidf)[::-1]
    top_words = []
    for idx in indices[:K]:
        top_words.append(features[idx])
    return top_words

In [0]:
top_english = top_keywords_classes(X_english, features_english, K=15)
print(np.array(top_english[:15]))
top_russian = top_keywords_classes(X_russian, features_russian, K=15)
print(np.array(top_russian[:15]))

['mower' 'crop' 'grass' 'rake' 'harvesting' 'harvester' 'deck' 'bale'
 'blade' 'trimmer' 'cutter' 'lawn' 'reel' 'stalk' 'arm']
['уборка' 'комбайн' 'транспортер' 'сельскохозяйственный' 'стебель'
 'барабан' 'убирать' 'рама' 'режущий' 'уборочный' 'элеватор' 'культура'
 'корень' 'пруток' 'машина']


In [0]:
muse_eng_words = list(src_word2id.keys())
muse_ru_words = list(tgt_word2id.keys())

In [0]:
#Сколько ключевых слов есть среди muse embeddings?
# с леммами - русских 80%, английских 100%

cnt = 0
found_eng = []
for word in set(top_english):
    if word in muse_eng_words:
        cnt+=1
        found_eng.append(word)

print(cnt/len(set(top_english))*100)       

100.0


In [0]:
cnt = 0
found_ru = []
for word in set(top_russian):
    if word in muse_ru_words:
        cnt+=1
        found_ru.append(word)

print(cnt/len(set(top_russian))*100)

80.0


In [0]:
def mean_top_vector(words, weights, lang, num_similar=1):
    '''
    words - топ ключевых слов текста
    weights - вес tfidf для каждого слова из списка
    lang - 'eng', 'ru' или 'ru_similar'
    num_similar - сколько похожих для каждого слова на русском искать
    для списка слова возвращает один вектор - взвешенная по tfidf сумма эмбеддингов каждого слова
    для русского два варианта: взвешивать muse embeddings для русского или сначала находить ближайшие на английском и 
    взвешивать их
    '''
    if lang == 'eng':  
        indices = [idx for idx, token in enumerate(words) if token in muse_eng_words]
        words = np.array(words)[indices]
        weights = np.array(weights)[indices].reshape(1,-1)
        cnt = len(words)
        muse = np.array([src_embeddings[src_word2id[token]] for token in words])
        vector = weights @ muse
    elif lang == 'ru':  
        indices = [idx for idx, token in enumerate(words) if token in muse_ru_words]
        words = np.array(words)[indices]
        weights = np.array(weights)[indices].reshape(1,-1)
        cnt = len(words)
        muse = np.array([tgt_embeddings[tgt_word2id[token]] for token in words])
        vector = weights @ muse
    else:
        indices = [idx for idx, token in enumerate(words) if token in muse_ru_words]
        words = np.array(words)[indices] 
        weights = np.array(weights)[indices].reshape(1,-1)
        cnt = len(words)
        if num_similar == 1:
            ind_best = get_similar(words, src_emb_norm, tgt_embeddings, tgt_id2word, K=num_similar).flatten()
            muse = src_embeddings[ind_best, :]
        else:
            ind_best = get_similar(words, src_emb_norm, tgt_embeddings, tgt_id2word, K=num_similar)
            muse = np.mean(src_embeddings[ind_best], axis = 1)
        vector = weights @ muse

    return vector/cnt

In [0]:
X_train, feat_train = matrix_for_features(train_lemm, labels=[])
X_val, feat_val = matrix_for_features(val_lemm, labels=[])
X_test, feat_test = matrix_for_features(test_lemm, labels=[])

In [0]:
X_train.shape[0]

68805

In [0]:
def matrix_for_classification(tfidf_matrix, feature_names, lang, num_similar=1):
    vectors = []
    for i in tqdm(range(tfidf_matrix.shape[0])):
        tfidf = tfidf_matrix[i].toarray()[0]
        indices_best = np.argsort(tfidf)[::-1][:15]
        vector = mean_top_vector(feature_names[indices_best], tfidf[indices_best], lang, num_similar=num_similar)    #tfidf[indices_best]
        vectors.append(vector)
    return np.array(vectors).squeeze(1)


In [0]:
train_matrix = matrix_for_classification(X_train, feat_train, 'eng')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
train_matrix.squeeze(1).shape

(68805, 300)

In [0]:
val_matrix = matrix_for_classification(X_val, feat_val, 'eng')
test_matrix = matrix_for_classification(X_test, feat_test, 'ru')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
test_matrix_similar = matrix_for_classification(X_test, feat_test, 'ru_similar', num_similar=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
np.save('/content/gdrive/My Drive/cross embeddings experiments/X_test_array_K5', test_matrix_similar)

In [0]:
test_similar5 = test_matrix_similar
test_similar1 = np.load('/content/gdrive/My Drive/cross embeddings experiments/X_test_array_K1.npy')
test_similar3 = np.load('/content/gdrive/My Drive/cross embeddings experiments/X_test_array_K3.npy')


#### Logistic regression

In [0]:
random.seed(13)
np.random.seed(13)
torch.manual_seed(13)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(13)
torch.cuda.manual_seed(13)

In [0]:
from tqdm import tnrange
def iterate_minibatches(data, labels, batch_size=100, shuffle=True, verbose=True):
    indices = np.arange(len(data))
    if shuffle:
        indices = np.random.permutation(indices)        
    irange = tnrange if verbose else range
 #   print(indices)
    for start in irange(0, len(indices), batch_size):
        yield data[indices[start: start + batch_size]], labels[indices[start: start + batch_size]]

In [0]:
class LogisticRegression(nn.Module):
    def __init__(self, emb_dim, num_labels):
        super().__init__()
        self.linear = nn.Linear(emb_dim, num_labels)
        nn.init.xavier_normal_(self.linear.weight)

    def forward(self, emb):
        return F.log_softmax(self.linear(emb), dim=1)

In [0]:
N_EPOCHS = 600
LEARNING_RATE = 4e-3
model = LogisticRegression(300,225).to(DEVICE) 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
#loss_function = nn.NLLLoss()
loss_function = nn.NLLLoss()

In [0]:
def train_model_epoch(train_dataset, labels, model, optimizer, batch_size):
    model.train()
    train_loss = 0
    train_predictions = np.array([])
    pred_labels = np.array([])
    
    for texts, labs in iterate_minibatches(np.array(train_dataset), np.array(labels), batch_size=batch_size, shuffle=True, verbose=False):
        emb_ix = torch.tensor(texts, dtype=torch.float32).to(DEVICE) #cuda
     #   print(labels[i: i+batch_size])
        res_labels = torch.tensor(labs).to(DEVICE)     #cuda 
      #  print(labels.shape)
        optimizer.zero_grad()
        output = model(emb_ix)
 
        loss = loss_function(output, res_labels)
        train_loss += loss.item()
   #     train_acc += (output.argmax(1) == res_labels).sum().item()
      #  print(train_acc)
        
        y_pred = output.argmax(1).cpu().numpy()
        train_predictions = np.hstack((train_predictions, y_pred))
        pred_labels = np.hstack((pred_labels, res_labels.cpu().numpy()))
     #   print(train_f1)
     #   break
        loss.backward()
        optimizer.step()

    train_f1_micro = f1_score(pred_labels, train_predictions, average='micro')*100
    train_f1_weighted = f1_score(pred_labels, train_predictions, average='weighted')*100

    return train_loss / (len(train_dataset)//batch_size), train_f1_micro, train_f1_weighted

In [0]:
def test(test_dataset, labels, model, batch_size):
    model.eval()
#    acc = 0
    test_predictions = np.array([])
    test_loss = 0
    for texts, labs in iterate_minibatches(np.array(test_dataset), np.array(labels), batch_size=batch_size, shuffle=False, verbose=False):
        with torch.no_grad():
            emb_ix = torch.tensor(texts, dtype = torch.float32).to(DEVICE) #cuda
            res_labels = torch.tensor(labs).to(DEVICE)   #cuda
        #  print(labels.shape)
            output = model(emb_ix)
            y_pred = output.argmax(1).cpu().numpy()
            loss = loss_function(output, res_labels)
            test_loss += loss.item()
            test_predictions = np.hstack((test_predictions, y_pred))

    test_f1_micro = f1_score(labels, test_predictions, average='micro')*100
    test_f1_weighted = f1_score(labels, test_predictions, average='weighted')*100


    return test_loss / (len(test_dataset)//batch_size), test_f1_micro, test_f1_weighted

In [0]:
print('Training...')
best_f1 = 0
max_step = 2
step=0
for epoch in tnrange(N_EPOCHS+1):
    train_loss, train_f1_micro, train_f1_weighted = train_model_epoch(train_matrix, y_train, model, optimizer, batch_size=300)
 #   train_loss, train_acc = train_model_epoch_sample(train_texts, labels, model, optimizer)
    val_loss, val_f1_micro, val_f1_weighted = test(val_matrix, y_val, model, batch_size=300)
    if val_f1_micro < best_f1 + 0.1:
      step += 1
      
      if step > max_step:
        break
    else:
      best_f1 = val_f1_micro
      step = 0
      weights_model = copy.deepcopy(model.state_dict())
           
    
      new_model = LogisticRegression(300,225).to(DEVICE)
      new_model.load_state_dict(weights_model)

    if epoch % 10 == 0:
     #   test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=300)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
     #   print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {val_f1_micro:.4f} | Val f1 weighted: {val_f1_weighted:.4f}')
print('Finished training.')

Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """



Train loss: 5.24823990 | Train f1 micro: 9.3874  | Train f1 weighted: 6.9598

Val f1 micro: 15.9269 | Val f1 weighted: 11.1718

Train loss: 3.97991458 | Train f1 micro: 29.7478  | Train f1 weighted: 24.4521

Val f1 micro: 29.4244 | Val f1 weighted: 24.1010

Train loss: 3.37353516 | Train f1 micro: 35.4495  | Train f1 weighted: 31.3817

Val f1 micro: 34.8925 | Val f1 weighted: 30.8584

Train loss: 3.02740513 | Train f1 micro: 38.7443  | Train f1 weighted: 35.5592

Val f1 micro: 37.5525 | Val f1 weighted: 34.3597

Train loss: 2.80880737 | Train f1 micro: 40.8866  | Train f1 weighted: 38.3521

Val f1 micro: 39.3148 | Val f1 weighted: 36.6850

Train loss: 2.66178766 | Train f1 micro: 42.5114  | Train f1 weighted: 40.4801

Val f1 micro: 40.7148 | Val f1 weighted: 38.6011

Train loss: 2.55492818 | Train f1 micro: 43.7483  | Train f1 weighted: 42.0709

Val f1 micro: 41.8595 | Val f1 weighted: 40.1480

Train loss: 2.47622732 | Train f1 micro: 44.7671  | Train f1 weighted: 43.3392

Val f1 micr

In [0]:
def compute_f1(model, data, labels):
    model.eval()
    emb_ix = torch.tensor(data, dtype = torch.float32).to(DEVICE)
    output = new_model(emb_ix)
    y_pred = output.argmax(1).cpu().numpy()
    print('f1_micro= ', np.round(f1_score(labels, y_pred, average='micro')*100, 2))
    print('f1_weighted= ', np.round(f1_score(labels, y_pred, average='weighted')*100, 2))

In [0]:
print('Train f1')
compute_f1(new_model, train_matrix, y_train)
print('Val f1')
compute_f1(new_model, val_matrix, y_val)
print('Test f1')
compute_f1(new_model, test_matrix, y_test)
print('Test f1 english embeddings, 1 similar')
compute_f1(new_model, test_similar1, y_test)
print('Test f1 english embeddings, 3 similar')
compute_f1(new_model, test_similar3, y_test)
print('Test f1 english embeddings, 5 similar')
compute_f1(new_model, test_similar5, y_test)

Train f1
f1_micro=  44.79
f1_weighted=  43.33
Val f1
f1_micro=  42.47
f1_weighted=  40.98
Test f1
f1_micro=  23.87
f1_weighted=  21.9
Test f1 english embeddings, 1 similar
f1_micro=  25.98
f1_weighted=  23.82
Test f1 english embeddings, 3 similar
f1_micro=  25.73
f1_weighted=  23.63
Test f1 english embeddings, 5 similar
f1_micro=  25.03
f1_weighted=  23.03
